## Notebook 1: Extracción de Datos
### Descripción General
Este notebook tiene como objetivo extraer los datos del dataset Bugs.jar. Para ello, accedemos a los repositorios Git, identificamos las ramas que contienen bugs, y extraemos los reportes y parches de cada bug.

### Sección 1: Importación de Librerías

In [5]:
import os
import git
import yaml
from unidiff import PatchSet
import pandas as pd


### Sección 2: Definición de Funciones

Función: process_project

In [15]:
# Esta función procesa un proyecto específico del dataset Bugs.jar.
# - Identifica todas las ramas que contienen bugs utilizando el prefijo 'bugs-dot-jar_'.
# - Hace checkout en cada rama para acceder a los archivos relevantes.
# - Lee el reporte del bug y el parche (diff) asociado.
# - Si los datos necesarios están presentes, almacena la información en una lista de diccionarios.

def process_project(project_path):
    repo = git.Repo(project_path)
    bug_branches = [ref.name for ref in repo.remote().refs if ref.name.startswith('origin/bugs-dot-jar_')]
    data = []

    for branch_name in bug_branches:
        print(f'Processing branch: {branch_name}')
        local_branch = branch_name.replace('origin/', '')
        try:
            # Crear o actualizar la rama local
            repo.git.checkout('-B', local_branch, branch_name)
        except Exception as e:
            print(f'Error checking out branch {branch_name}: {e}')
            continue

        # Después de hacer checkout, el directorio .bugs-dot-jar debería estar en project_path
        bug_dir = os.path.join(project_path, '.bugs-dot-jar')

        # Leer archivos necesarios
        bug_report = read_bug_report(bug_dir)
        patch_set = read_patch_set(bug_dir)

        if bug_report and patch_set:
            # Procesar los datos y añadir a la lista 'data'
            # (Tu código existente para procesar y extraer la información)
            data.append({
                'project': os.path.basename(project_path),
                'branch': local_branch,
                'bug_report': bug_report,
                # Agrega otros campos necesarios
            })
        else:
            print(f'Skipping branch {branch_name} due to missing data.')

    return data
# 1. Se accede al repositorio del proyecto mediante Git.
# 2. Se filtran las ramas que comienzan con 'origin/bugs-dot-jar_'.
# 3. Por cada rama:
#    - Se intenta hacer checkout en la rama correspondiente.
#    - Se accede al directorio '.bugs-dot-jar' que contiene los archivos necesarios.
# 4. Si se encuentran tanto el reporte del bug como el parche:
#    - Se almacena la información en un diccionario y se añade a la lista de resultados.


In [7]:
# Función: `read_bug_report`
# - Lee el archivo 'bug-report.yml' que contiene la información del bug.
# - Si el archivo no existe o está vacío, muestra un mensaje de advertencia.
# - Si el archivo se lee correctamente, se devuelve un diccionario con los datos del bug.

def read_bug_report(bug_dir):
    bug_report_path = os.path.join(bug_dir, 'bug-report.yml')
    if os.path.exists(bug_report_path):
        try:
            with open(bug_report_path, 'r', encoding='utf-8') as f:
                content = f.read()
                if not content.strip():
                    print(f'Bug report file is empty: {bug_report_path}')
                    return None
                return yaml.safe_load(content)
        except Exception as e:
            print(f'Error reading bug report {bug_report_path}: {e}')
            return None
    else:
        print(f'Bug report file does not exist: {bug_report_path}')
        return None
# 1. Si el archivo 'bug-report.yml' no existe:
#    - Se imprime un mensaje de advertencia.
# 2. Si el archivo está vacío:
#    - Se ignora el archivo y se devuelve None.
# 3. Si hay un error al leer el archivo:
#    - Se imprime el error y se devuelve None.

def read_patch_set(bug_dir):
    patch_path = os.path.join(bug_dir, 'developer-patch.diff')  # Ajuste del nombre del archivo
    if os.path.exists(patch_path):
        try:
            with open(patch_path, 'r', encoding='utf-8') as f:
                content = f.read()
                if not content.strip():
                    print(f'Patch file is empty: {patch_path}')
                    return None
                return PatchSet(content)
        except Exception as e:
            print(f'Error reading patch file {patch_path}: {e}')
            return None
    else:
        print(f'Patch file does not exist: {patch_path}')
        return None
# Función: `read_patch_set`
# - Lee el archivo 'developer-patch.diff' que contiene el parche asociado al bug.
# - Si el archivo no existe o está vacío, muestra un mensaje de advertencia.
# - Si se lee correctamente, se devuelve un PatchSet que representa los cambios.


In [8]:
def main():
    base_path = 'bugs-jar' 
    projects = ['accumulo', 'camel', 'commons-math', 'flink', 'jackrabbit-oak', 'logging-log4j2', 'maven', 'wicket']
    all_data = []

    for project_name in projects:
        project_path = os.path.join(base_path, project_name)
        print(f'Processing project: {project_name}')
        project_data = process_project(project_path)
        all_data.extend(project_data)

    # Convertir a DataFrame y guardar
    df = pd.DataFrame(all_data)
    df.to_csv('bugs_data1.csv', index=False)
    print('Data extraction complete. Saved to bugs_data1.csv.')

if __name__ == '__main__':
    main()


Processing project: accumulo
Processing branch: origin/bugs-dot-jar_ACCUMULO-1044_9396979b
Processing branch: origin/bugs-dot-jar_ACCUMULO-1044_ea2f9856
Processing branch: origin/bugs-dot-jar_ACCUMULO-1051_25cf3ccd
Processing branch: origin/bugs-dot-jar_ACCUMULO-1120_474b2577
Processing branch: origin/bugs-dot-jar_ACCUMULO-1183_742960f1
Processing branch: origin/bugs-dot-jar_ACCUMULO-1183_cfbf5999
Processing branch: origin/bugs-dot-jar_ACCUMULO-1190_e29dc4f5
Processing branch: origin/bugs-dot-jar_ACCUMULO-1192_9476b877
Processing branch: origin/bugs-dot-jar_ACCUMULO-1192_c489d866
Processing branch: origin/bugs-dot-jar_ACCUMULO-1199_813109d7
Processing branch: origin/bugs-dot-jar_ACCUMULO-1312_d9ab8449
Processing branch: origin/bugs-dot-jar_ACCUMULO-1348_6ff92b12
Processing branch: origin/bugs-dot-jar_ACCUMULO-1348_ef0f6ddc
Processing branch: origin/bugs-dot-jar_ACCUMULO-1358_4d10c92f
Processing branch: origin/bugs-dot-jar_ACCUMULO-1358_6c565dfb
Processing branch: origin/bugs-dot-jar_AC

### Sección: Normalización y Expansión del DataFrame
Descripción General
Este bloque de código se encarga de:

Cargar los datos desde un archivo CSV.
Expandir los datos anidados en la columna bug_report para extraer los detalles del reporte de bugs.
Combinar los datos originales con los nuevos campos extraídos.
Guardar el DataFrame expandido en un nuevo archivo CSV para su posterior análisis.

In [12]:
# Re-import the CSV to ensure the DataFrame is loaded correctly
file_path = "bugs_data1.csv"
df = pd.read_csv(file_path)

# Normalize the 'bug_report' column to extract nested details into separate columns
df_bug_details = pd.json_normalize(df['bug_report'].apply(eval))  # Convert string representation of dict to dict

# Combine the normalized data with the original DataFrame, dropping the original 'bug_report' column
df_combined = pd.concat([df.drop('bug_report', axis=1), df_bug_details], axis=1)

# Save the expanded DataFrame to a new CSV
expanded_file_path = "bugs_data_expanded.csv"
df_combined.to_csv(expanded_file_path, index=False)

expanded_file_path


'bugs_data_expanded.csv'

In [ ]:
file_path = "bugs_data_expanded.csv"
df = pd.read_csv(file_path)

In [14]:
df

,project,branch,BugID,Summary,Description
0,accumulo,bugs-dot-jar_ACCUMULO-1044_9396979b,ACCUMULO-1044,bulk imported files showing up in metadata aft...,Bulk import fails. The file is moved to the f...
1,accumulo,bugs-dot-jar_ACCUMULO-1044_ea2f9856,ACCUMULO-1044,bulk imported files showing up in metadata aft...,Bulk import fails. The file is moved to the f...
2,accumulo,bugs-dot-jar_ACCUMULO-1051_25cf3ccd,ACCUMULO-1051,Authorizations has inconsistent serialization,The same set of authorizations may not seriali...
3,accumulo,bugs-dot-jar_ACCUMULO-1120_474b2577,ACCUMULO-1120,stop-all doesn't work: Error BAD_CREDENTIALS f...,{noformat}\n$ bin/accumulo admin stopAll\n2013...
4,accumulo,bugs-dot-jar_ACCUMULO-1183_742960f1,ACCUMULO-1183,ProxyServer does not set column information on...,The createScanner method uses the options from...
...,...,...,...,...,...
1153,wicket,bugs-dot-jar_WICKET-5966_d547fcd4,WICKET-5966,ResourceUtils.getLocaleFromFilename can't hand...,I think the ResourceUtils.getLocaleFromFilenam...
1154,wicket,bugs-dot-jar_WICKET-5968_8b7946d8,WICKET-5968,CachingResourceLocator lookup key doesn't take...,CachingResourceLocator uses a CacheKey to stor...
1155,wicket,bugs-dot-jar_WICKET-5980_294b0b2f,WICKET-5980,When using Servlet 3.0 filter Wicket calculate...,When using a servlet 3.0 filter with annotatio...
1156,wicket,bugs-dot-jar_WICKET-5981_eb125865,WICKET-5981,Significant Performance Degradation From Wicke...,I am experiencing a significant performance de...
